In [1]:
import numpy as np
import pandas as pd
import sys
import os
import re

### This notebook:
* Opens `censat_table_diploid_batch1.csv` created by `batch1/hmm_flagger/make_hmm_flagger_data_tables.ipynb`
* Takes read paths from tables 
    * `batch1/hmm_flagger/read_tables/ont_reads_table.csv`
    * `batch1_jan_12_2025/hmm_flagger/read_tables/hifi_full_reads_table.jan_12_2025.csv` 
* Makes separate data tables for HiFi and ONT new runs that were missed in the previous runs for batch 1
* Some HiFi runs were missed because I used only the DeepConsensus table but now I'm using a table with additional Revio data
* Some ONT runs were missed because some samples had 'GM' prefix instead of 'NA' prefix
* Saves the final data tables in `hifi/` and `ont/` subdirectories and they will be used for creating input json files

In [2]:
BASE_DIR="/private/groups/hprc/qc_hmm_flagger/hprc_intermediate_assembly/assembly_qc"

In [3]:
censat_table_diploid = pd.read_csv(f'{BASE_DIR}/batch1/hmm_flagger/censat_table_diploid_batch1.csv')

In [4]:
# parse hifi reads table (jan 12 2025 version containing both Revio and DeepConsensus)
hifi_full_reads_table = pd.read_csv(f'read_tables/hifi_full_reads_table.jan_12_2025.csv')

# parse hifi reads table (DeepConsensus only)
hifi_dc_reads_table = pd.read_csv(f'{BASE_DIR}/batch1/hmm_flagger/read_tables/hifi_reads_table.csv')

In [5]:
# here I will take only the samples with less than 40x HiFi_DC coverage
# so if in the new table we have higher coverage we will rerun it with greater than 40x coverage
all_samples_dc_cov_lt_40   = set(hifi_dc_reads_table["sample_id"][hifi_dc_reads_table["total_coverage"] < 40])
all_samples_full_cov_gt_40 = set(hifi_full_reads_table["sample_id"][hifi_full_reads_table["total_coverage"] > 40])

samples_to_be_rerun_higher_cov = all_samples_full_cov_gt_40.intersection(all_samples_dc_cov_lt_40)

print('There are',
      len(samples_to_be_rerun_higher_cov),
      'samples with coverage > 40x in the new table but not in the old table')

There are 40 samples with coverage > 40x in the new table but not in the old table


In [6]:
# here I will take the new samples that were absent from HiFi_DC table
all_samples_dc = set(hifi_dc_reads_table["sample_id"])
all_samples_full = set(hifi_full_reads_table["sample_id"])

samples_to_be_run = all_samples_full.difference(all_samples_dc)

print('There are',
      len(samples_to_be_run),
      'samples existing in the new table but not in the old table')

There are 107 samples existing in the new table but not in the old table


In [7]:
all_samples_for_new_table = samples_to_be_run.union(samples_to_be_rerun_higher_cov)
# make a new table with only samples that need to be run (or rerun with higher coverage)
hifi_full_reads_table_new_only = hifi_full_reads_table[hifi_full_reads_table['sample_id'].isin(all_samples_for_new_table)]

In [8]:
# merge with censat table
hifi_data_table_jan_12_2025 = pd.merge(censat_table_diploid, hifi_full_reads_table_new_only, on='sample_id',  how='inner')
len(hifi_data_table_jan_12_2025)

83

In [9]:
hifi_data_table_first_run = pd.read_csv(f'{BASE_DIR}/batch1/hmm_flagger/hifi/hmm_flagger_hifi_data_table.csv')

In [10]:
sample_ids_censat_batch1 = set(censat_table_diploid['sample_id'])
sample_ids_first_run = set(hifi_data_table_first_run['sample_id'])
sample_id_jan_12_2025 = set(hifi_data_table_jan_12_2025['sample_id'])
missing_samples = sample_ids_censat_batch1.difference(sample_ids_first_run.union(sample_id_jan_12_2025))
print(f"There are {len(missing_samples)} missing samples without HiFi reads for censat batch 1")

There are 0 missing samples without HiFi reads for censat batch 1


In [11]:
os.makedirs("hifi", exist_ok=True)
hifi_data_table_jan_12_2025.to_csv('hifi/hmm_flagger_hifi_data_table.csv', index=False)

## Make a table for ONT data with 'GM' prefix replaced with 'NA'

In [12]:
ont_reads_table = pd.read_csv(f'{BASE_DIR}/batch1/hmm_flagger/read_tables/ont_reads_table.csv')

In [13]:
ont_reads_table_with_GM = ont_reads_table[ont_reads_table['sample_id'].str.startswith('GM')]
ont_reads_table_with_GM.head()

,sample_id,read_files_downsampled,number_of_read_files_downsampled,total_coverage_downsampled,number_of_cores_per_task_downsampled,mapper_preset,kmer_size,read_files,number_of_read_files,total_coverage,coverage,number_of_cores_per_task,sequencing_chemistry,hmm_flagger_window_size,hmm_flagger_alpha_tsv
0,GM18522,['s3://human-pangenomics/working/HPRC/NA18522/...,3,69.01,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18522/...,3,69.01,"[23.22, 29.67, 16.12]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
1,GM18570,['s3://human-pangenomics/working/HPRC/NA18570/...,3,66.22,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18570/...,3,66.22,"[23.54, 23.88, 18.8]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
2,GM18612,['s3://human-pangenomics/working/HPRC/NA18612/...,3,72.34,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18612/...,3,72.34,"[24.45, 19.88, 28.01]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
3,GM18747,['s3://human-pangenomics/working/HPRC/NA18747/...,3,75.12,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18747/...,3,75.12,"[24.51, 26.08, 24.53]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
4,GM18971,['s3://human-pangenomics/working/HPRC/NA18971/...,3,79.88,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18971/...,3,79.88,"[24.8, 28.49, 26.59]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...


In [14]:
ont_reads_table_with_GM.loc[:,'sample_id'] = ont_reads_table_with_GM['sample_id'].str.replace('GM', 'NA')
ont_reads_table_with_GM.head()

/private/home/masri/apps/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,sample_id,read_files_downsampled,number_of_read_files_downsampled,total_coverage_downsampled,number_of_cores_per_task_downsampled,mapper_preset,kmer_size,read_files,number_of_read_files,total_coverage,coverage,number_of_cores_per_task,sequencing_chemistry,hmm_flagger_window_size,hmm_flagger_alpha_tsv
0,NA18522,['s3://human-pangenomics/working/HPRC/NA18522/...,3,69.01,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18522/...,3,69.01,"[23.22, 29.67, 16.12]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
1,NA18570,['s3://human-pangenomics/working/HPRC/NA18570/...,3,66.22,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18570/...,3,66.22,"[23.54, 23.88, 18.8]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
2,NA18612,['s3://human-pangenomics/working/HPRC/NA18612/...,3,72.34,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18612/...,3,72.34,"[24.45, 19.88, 28.01]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
3,NA18747,['s3://human-pangenomics/working/HPRC/NA18747/...,3,75.12,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18747/...,3,75.12,"[24.51, 26.08, 24.53]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...
4,NA18971,['s3://human-pangenomics/working/HPRC/NA18971/...,3,79.88,21,map-ont,15,['s3://human-pangenomics/working/HPRC/NA18971/...,3,79.88,"[24.8, 28.49, 26.59]",21,R941,16000,https://raw.githubusercontent.com/mobinasri/fl...


In [15]:
# merge with censat table
ont_data_table_jan_12_2025 = pd.merge(censat_table_diploid, ont_reads_table_with_GM, on='sample_id',  how='inner')
len(ont_data_table_jan_12_2025)

13

In [16]:
ont_data_table_first_run = pd.read_csv(f'{BASE_DIR}/batch1/hmm_flagger/ont/hmm_flagger_ont_data_table.csv')

In [17]:
sample_ids_censat_batch1 = set(censat_table_diploid['sample_id'])
sample_ids_first_run = set(ont_data_table_first_run['sample_id'])
sample_id_jan_12_2025 = set(ont_data_table_jan_12_2025['sample_id'])
missing_samples = sample_ids_censat_batch1.difference(sample_ids_first_run.union(sample_id_jan_12_2025))
print(f"There are {len(missing_samples)} missing samples without ONT reads for censat batch 1")
print("\n".join(missing_samples))

There are 11 missing samples without ONT reads for censat batch 1
NA21309
HG02055
HG02818
HG03486
HG02723
HG01109
NA18906
HG02080
HG03098
NA20129
HG02109


In [18]:
# add column "suffix_mapping"
ont_data_table_jan_12_2025["suffix_mapping"] = ont_data_table_jan_12_2025["sequencing_chemistry"] + "_minimap2_2.28"

In [19]:
os.makedirs("ont", exist_ok=True)
ont_data_table_jan_12_2025.to_csv('ont/hmm_flagger_ont_data_table.csv', index=False)